In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
%pwd

'/dataLarihs/rayen/projects/Oysters'

In [3]:
train_df = pd.read_csv("sampled_train10.csv" , index_col = 0)
train_df

,sequence,ID,label
0,0.410940,A1,1
1,0.412114,A1,1
2,0.406828,A1,1
3,0.448529,A1,1
4,0.435020,A1,1
...,...,...,...
17387995,0.470848,I2,0
17387996,0.471435,I2,0
17387997,0.471435,I2,0
17387998,0.471435,I2,0


In [4]:
import pandas as pd

# Step 1: Create an empty DataFrame with the same column structure as train_df
expanded_df = pd.DataFrame(columns=train_df.columns)

# Step 2: Iterate over each unique ID
for unique_id in train_df['ID'].unique():
    # Step 3: Get the rows corresponding to the current ID
    id_rows = train_df.loc[train_df['ID'] == unique_id]
    
    # Step 4: Divide the current ID into the desired number of segments
    segments = 16
    
    # Calculate segment length
    segment_length = len(id_rows) // segments
    
    # Step 5: Create new rows by combining the divided IDs with the original gender and channel values
    for i in range(segments):
        segment_start = i * segment_length
        segment_end = (i + 1) * segment_length
        new_id = f"{unique_id}_{i+1}"
        new_rows = id_rows.iloc[segment_start:segment_end].copy()
        new_rows['ID'] = new_id
        expanded_df = pd.concat([expanded_df, new_rows])

# Reset the index of the expanded DataFrame
expanded_df.reset_index(drop=True, inplace=True)

expanded_df = expanded_df.astype(train_df.dtypes)

# Print the expanded DataFrame
train_df = expanded_df


In [5]:
expanded_df.dtypes

sequence    float64
ID           object
label         int64
dtype: object

In [6]:
grouped = train_df.groupby("ID")
grouped.size()

ID
A1_1     36187
A1_10    36187
A1_11    36187
A1_12    36187
A1_13    36187
         ...  
I2_5     36375
I2_6     36375
I2_7     36375
I2_8     36375
I2_9     36375
Length: 480, dtype: int64

In [7]:
min_count = grouped.size().min()
min_count

36187

In [8]:
grouped

In [9]:
from sklearn.preprocessing import LabelEncoder
import tqdm
import gc

In [ ]:
# Preprocess the data
continuous_features = ['sequence']

# Encode gender labels
y = train_df.groupby('ID')['label'].first().values

IDs = train_df.ID.unique()

sequences = []
current_sequence = []

for id in tqdm.tqdm(IDs) :
    current_sequence = train_df[train_df["ID"] == id][continuous_features].values
    #print(f"{id} s sequence length : {len(current_sequence)}")
    sequences.append(current_sequence[:min_count])
    #print(type(current_sequence))

 23%|████████████████████████████▋                                                                                               | 111/480 [00:47<02:37,  2.34it/s]

In [ ]:
len(sequences[0].flatten())

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

epochs = 1000
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
y

In [ ]:
len(sequences[2])

In [ ]:
X = torch.tensor(np.array(sequences), dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
y

In [ ]:
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
"""eval_df = pd.read_csv("Datasets/Gender_data/val_df.csv" , index_col = 0)
grouped = eval_df.groupby("ID")
min_count = grouped.size().min()
sampled_dfs = []
for _ , data in grouped :
    sampled_dfs.append(data.head(min_count))
balanced_df = pd.concat(sampled_dfs).reset_index(drop= True)
# Encode gender labels
y_eval = label_encoder_2.transform(balanced_df.groupby('ID')['gender'].first().values)

IDs = balanced_df.ID.unique()

sequences = []
current_sequence = []

for id in IDs :
    current_sequence = balanced_df[balanced_df["ID"] == id][continuous_features].values
    print(f"{id} s sequence length : {len(current_sequence)}")
    sequences.append(current_sequence)
    print(type(current_sequence))
    
X_eval = torch.tensor(np.array(sequences), dtype=torch.float32)
y_eval = torch.tensor(y_eval, dtype=torch.float32)"""

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Only take the last output of the sequence
        out = self.sigmoid(out)
        return out

input_size = X.shape[2]
hidden_size = 64
output_size = 1

model = RNNModel(input_size, hidden_size, output_size)
model.to(device)

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss since the output is a single sigmoid value
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_loss = float('inf')  # Initialize the best loss as infinity
best_model_path = 'best_model.pt'  # Define the path to save the best model

for epoch in range(epochs):
    for inputs, targets in dataloader:

        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets.view(len(targets), 1))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        model.eval()
        print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {loss.item()}')

        # Save the model if there is an improvement in validation loss
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(model.state_dict(), best_model_path)
            print("Saving the best model...")
        model = model.to(device)
        model.train()





    

In [ ]:
# Load the best model for prediction
best_model = RNNModel(input_size, hidden_size, output_size)
best_model.load_state_dict(torch.load(best_model_path))
best_model.to("cpu")

with torch.no_grad():
    model = model.to("cpu")
    model.eval()
    outputs = model(X.to("cpu"))
    print(outputs)
    predicted = torch.round(outputs.squeeze())  # Round the sigmoid output to get binary predictions
    accuracy = (predicted == y).sum().item() / len(y)
    print(f'Accuracy: {accuracy * 100}%')